<a href="https://colab.research.google.com/github/clizarraga-UAD7/Notebooks/blob/main/IntroCOPC_LIDARPointCloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to COPC Data 

The [USGS](https://www.usgs.gov/products/data) is now offering a [3DEP](https://www.usgs.gov/3d-elevation-program) (3D Elevation Program) [Lidar Point Cloud COPC](https://planetarycomputer.microsoft.com/dataset/3dep-lidar-copc#overview) Data ([Cloud Optimized Point Cloud](https://lidarmag.com/2021/12/27/cloud-native-geospatial-lidar-with-the-cloud-optimized-point-cloud/)).

COPC - Cloud Optimized Point Cloud data files allow applications to select data for a window or a resolution, and allow them to limit how much data they must fetch, decompress, and process.

[COPC](https://copc.io/) files are compressed with the [LASzip algorithm](https://rapidlasso.de/laszip/) and stored as a [clustered octree](https://en.wikipedia.org/wiki/Octree) data sctructure.

To work with COPC data we need to install the following additional Python Libraries:
* [`PDAL`](https://pdal.io/en/latest/about.html),  The _Point Data Abstraction Library_ allows to translating and processing Point Cloud data. PDAL is not limited to [LIDAR](https://en.wikipedia.org/wiki/Lidar) Data. PDAL supports reading and writing COPC with [`readers.copc`](https://pdal.io/stages/readers.copc.html) and [`writers.copc`](https://pdal.io/en/latest/stages/writers.copc.html).
* [`PDAL-Python`](https://github.com/PDAL/python/). PDAL Python will help construct [PDAL pipelines](https://pdal.io/en/latest/pipeline.html).
* [`pystac_client`](https://pystac-client.readthedocs.io/en/stable/). Python package for working with [STAC Catalogs](https://www.stacindex.org/catalogs)
* [`planetary_computer`](https://planetarycomputer.microsoft.com). The Microsoft Planetary Computer Library API. 
* [`PIL`](https://pypi.org/project/Pillow/). Python Image Libary (Pillow).
* [`pyproj`](https://pypi.org/project/pyproj/). Python interface to [`PROJ`](https://proj.org/) (cartographic projections and coordinate transformations library).
* [shapely](https://pypi.org/project/shapely/). Manipulation and analysis of geometrical objects in the cartesian plane.
* [geopandas](https://pypi.org/project/geopandas/). Adds geographical data support to the [Pandas](https://pandas.pydata.org) data analysis library.
* [mapclassify](https://pypi.org/project/mapclassify/). Maps classification support. 


We install all the required libraries not included in Google Colab base libraries.


In [ ]:
# Install pystac_client
!pip install pystac_client --quiet

     |████████████████████████████████| 146 kB 6.9 MB/s 
     |████████████████████████████████| 62 kB 558 kB/s 


In [ ]:
# Install planetary_computer
!pip install planetary_computer --quiet


In [ ]:
# Install Python Image Library manipulation
!pip install pillow --quiet


In [ ]:
# Install pyproj
!pip install pyproj --quiet


     |████████████████████████████████| 6.3 MB 5.1 MB/s 


In [ ]:
# Install Shapely
!pip install shapely --quiet


In [ ]:
# Install Geopandas
!pip install geopandas --quiet


     |████████████████████████████████| 1.0 MB 5.2 MB/s 
     |████████████████████████████████| 16.7 MB 333 kB/s 


In [ ]:
# Installl mapclassify
!pip install mapclassify --quiet


In [ ]:
# Import the required libraries
#import pdal
import pystac_client
import planetary_computer
import PIL
import pyproj

import folium
import matplotlib
import mapclassify 


Next function estimates the UTM zone of a point using PyPROJ methods.

In [ ]:
# Estimate our UTM zone
from pyproj import CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info


def get_utm(point):
    longitude, latitude = point.x, point.y
    buffer = 0.001
    utm_crs_list = query_utm_crs_info(
        datum_name="WGS 84",
        area_of_interest=AreaOfInterest(
            west_lon_degree=longitude - buffer,
            south_lat_degree=latitude - buffer,
            east_lon_degree=longitude + buffer,
            north_lat_degree=latitude + buffer,
        ),
    )
    utm_crs = CRS.from_epsg(utm_crs_list[0].code)
    return utm_crs


Next, select a point in a map where to query. A GeoJSON point geometry is is defined by the `bean` variable, with a 400 m buffer. After this, we need to reproject back to EPSG:4326 coordinates so we can query the STAC API and finally plot to see what we will do.

In [ ]:
# Prescott AZ map (Select other LONG,LAT coordinates)
bean = {"type": "Point", "coordinates": [-112.468611, 34.54]}

from shapely.geometry import shape
from shapely.ops import transform

geom = shape(bean)

utm = get_utm(geom)

wgs84 = pyproj.CRS("EPSG:4326") # World Geodetic System 84, same as used by the GPS. 

project_dd_to_utm = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
project_utm_to_dd = pyproj.Transformer.from_crs(utm, wgs84, always_xy=True).transform

utm_point = transform(project_dd_to_utm, geom)
window = utm_point.buffer(400)

window_dd = transform(project_utm_to_dd, window)

import geopandas

df = geopandas.GeoDataFrame(geometry=[window_dd], crs="EPSG:4326")

df.explore()


**Query the Planetary Computer STAC API.** 

Query the STAC API for the `3dep-lidar-copc` collections that intersects with the University of Arizona polygon shown above. 

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)

search = catalog.search(collections=["3dep-lidar-copc"], intersects=window_dd)
ic = search.get_all_items()


In [ ]:
signed = planetary_computer.sign(ic)


Define some variables used for querying

In [ ]:
OUTPUT_RESOLUTION = 2.0
READ_RESOLUTION = 2.0
polygon = window.wkt + f" / EPSG:{utm.to_epsg()}"


In [ ]:
!pip install copclib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 743 kB 5.0 MB/s 


In [ ]:
import copclib as copc

# Create a reader object
URL="https://www.sciencebase.gov/catalog/item/6113c8d3d34ed11898f7e03a.laz"
reader = copc.FileReader(URL)

# Get the node metadata from the hierarchy
node = reader.FindNode(copc.VoxelKey(0, 0, 0, 0))
# Fetch the points of a node
points = reader.GetPoints(node)

# Iterate through each point
for point in points:
    print(point.x, point.y, point.z)


RuntimeError: ignored

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 743 kB 24.4 MB/s 
